# DeepFactor - Experiments

### Deep Factor (Wang et al.,  2019)

Hybrid method that uses *local* methods that are specific to each time series in the dataset as well as a *global* representation trained on the entire data. The latter is a RNN to learn universal pattern. The current version uses another RNN as local model -> **DeepFactor-RNN (DF-RNN)**

- [`DeepFactorEstimator`](http://gluon-ts.mxnet.io/master/api/gluonts/gluonts.model.deep_factor.html?highlight=deepfactorestimator#gluonts.model.deep_factor.DeepFactorEstimator)
- [Wang et al. (2019)](https://arxiv.org/abs/1905.12417)

In [1]:
# if version does not include DeepState, uncomment the following:
# !pip install git+https://github.com/awslabs/gluon-ts.git

In [1]:
import mxnet
mxnet.__version__

'1.4.1'

In [2]:
import gluonts
gluonts.__version__

'0.4.1'

In [3]:
import pprint
from functools import partial

# standard imports
import numpy as np
import pandas as pd

# vis imports 
%matplotlib inline
import matplotlib.pyplot as plt

# json
import json

# gluon data 
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# gluon distribution
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.distribution.gaussian import GaussianOutput

# model imports 
# from gluonts.model.deepstate import DeepStateEstimator
from gluonts.model.deep_factor import DeepFactorEstimator
# from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

INFO:root:Using CPU


In [4]:
def main(seed=42, data="m4_daily", epochs=100, batches=50):
    
    mx.random.seed(seed)
    np.random.seed(seed)
    
    dataset = get_dataset(data)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,
        num_batches_per_epoch=batches,
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepFactorEstimator(
        trainer=trainer,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         num_layers_global=2,
#         num_layers_local=2
    )

    predictor = estimator.train(dataset.train)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    df = pd.DataFrame([output])

    return(df)

# if __name__ == "__main__":
#     simple_main()

In [18]:
%%time
if __name__ == "__main__":
    df = main(seed=42, data="m4_monthly", epochs=100, batches=100)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:08<00:00, 11.73it/s, avg_epoch_loss=2.69e+7]
INFO:root:Epoch[0] Elapsed time 8.566 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=26857936.586667
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:06<00:00, 14.97it/s, avg_epoch_loss=1.28e+7]
INFO:root:Epoch[1] Elapsed time 6.748 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=12783945.862222
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:05<00:00, 17.33it/s, avg_epoch_loss=2.6e+6]
INFO:root:Epoch[2] Elapsed time 5.778 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=2596963.907222
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:06<00:00, 15.27it/s, avg_epoch_loss=1.08e+6]
INFO:root:Epoch[3] Elapsed time 6.558 seconds
INFO:root:Epoch[3] Evaluat

KeyboardInterrupt: 

In [19]:
%%time
if __name__ == "__main__":
    df = main(seed=42, data="m4_monthly", epochs=250, batches=100)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:05<?, ?it/s, avg_epoch_loss=2.87e+7]


KeyboardInterrupt: 

In [20]:
%%time
if __name__ == "__main__":
    df = main(seed=42, data="m4_monthly", epochs=500, batches=100)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [21]:
%%time
if __name__ == "__main__":
    df = main(seed=42, data="m4_monthly", epochs=1000, batches=100)
df

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Epoch[0] Learning rate is 0.001
  0%|          | 0/100 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [22]:
%%time
if __name__ == "__main__":
    df = main(seed=42, data="m4_monthly", epochs=2000, batches=100)
df

KeyboardInterrupt: 

In [23]:
!jupyter nbconvert --output-dir="./html_outputs" --to html DeepFactor01-DeepFactor_experiments_with_gluonts.ipynb

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
print("Done!")